In [1]:
# push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
# push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
# push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/util"));
# ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Distributions

include("COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

In [3]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI"]

start_date = Date(2020, 5, 1)
end_date   = Date(2020, 5, 30)

pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 14;

In [4]:
N = length(states);
T = (end_date - start_date).value + 1;

In [5]:
forecast_admitted = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [6]:
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)[:];

In [7]:
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [8]:
beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available);
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [9]:
model = patient_redistribution(
    beds,
    forecast_initial,
    forecast_discharged,
    forecast_admitted,
    adj,
    Exponential(5)
)




Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-21


A JuMP Model
Minimization problem with:
Variables: 3300
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 600 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 300 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 1320 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1980 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: active_patients, overflow, sent

In [10]:
model = patient_redistribution(
    beds,
    forecast_initial,
    forecast_discharged,
    forecast_admitted,
    adj,
    Exponential(100),

    sent_penalty=0,
    smoothness_penalty=0,
    sendreceive_gap=0,
    min_send_amt=0,
    
    setup_cost=0,
    
    verbose=true,
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-21
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 22.6.0 22G513)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 900 rows, 3300 columns and 165900 nonzeros
Model fingerprint: 0xc4249c0e
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+04]
Presolve removed 24 rows and 1328 columns
Presolve time: 0.04s
Presolved: 876 rows, 1972 columns, 102487 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...


Barrier performed 0 iterations in 0.06 seconds (0.06 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iteration    Objective       Primal Inf.    Dual Inf.      Time
     172    2.7870404e+02 

0.071s
objective function value: 

278.704


In [11]:
results = PatientAllocationResults.results_all(sent, beds, forecast_initial, forecast_admitted, forecast_discharged, states, start_date, hospitalized_days);

In [12]:
println("Total overflow: ", results.total_overflow)
println("Average load: ", results.average_load)

Total overflow: 79776.95


Average load: 7.654549


In [13]:
results.summary_table

Row,location,total_sent,total_received,overflow,overall_load,overflow_nosent,overall_load_nosent
,String,Float64,Float64,Float64,Float64,Float32,Float32
1,CT,1869.54,538.009,13563.0,0.273449,13884.9,0.512099
2,DE,370.408,424.639,0.0,-0.188552,0.0,-0.149694
3,MA,772.159,891.608,0.0,0.0842814,0.0,0.0629188
4,MD,2083.36,827.46,0.0,-0.130556,73.7739,-0.00288614
5,ME,12.2474,124.016,0.0,0.0402802,0.0,-0.0101372
6,NH,242.458,128.235,0.0,-0.360967,0.0,-0.267712
7,NJ,2215.07,1682.03,42262.1,0.54747,49083.8,0.588553
8,NY,66.7645,4676.76,48397.0,0.689062,16734.5,0.482961
9,PA,2092.32,783.624,0.0,-0.0800303,0.0,0.0104626


In [14]:
results.sent_matrix_table

Row,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,88.3191,166.176,0.0,124.016,128.235,403.696,881.039,78.0586,0.0
2,DE,0.0,0.0,0.0,19.2845,0.0,0.0,80.9315,270.192,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,0.0,192.222,465.309,0.0,114.628
4,MD,0.0,302.847,0.0,0.0,0.0,0.0,313.578,761.365,705.565,0.0
5,ME,8.13678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.11058
6,NH,97.8416,0.0,91.1922,0.0,0.0,0.0,0.0,0.0,0.0,53.4247
7,NJ,197.419,0.0,634.24,540.09,0.0,0.0,0.0,761.871,0.0,81.4498
8,NY,34.4098,32.3547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,PA,200.202,1.11779,0.0,268.086,0.0,0.0,562.061,1060.86,0.0,0.0


In [17]:
println("First day:")
filter(row -> row.date == start_date, results.complete_table)

First day:


Row,location,date,sent,received,new_patients,active_patients,active_patients_nosent,capacity,overflow,load,overflow_nosent,load_nosent,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float32,Float64,Float64,Float32,Float32,Any,Any
1,CT,2020-05-01,0.0,0.0,373.65,1973.54,1973.54,1840.25,133.287,1.07243,133.287,1.07243,[],[]
2,DE,2020-05-01,0.0,0.0,23.0751,169.856,169.856,506.75,0.0,0.335187,0.0,0.335187,[],[]
3,MA,2020-05-01,0.0,18.0104,346.806,1502.72,1484.71,3997.75,0.0,0.375892,0.0,0.371387,[],"[(""NH"", 18.0104)]"
4,MD,2020-05-01,0.0,0.0,290.597,1772.15,1772.15,2228.75,0.0,0.79513,0.0,0.79513,[],[]
5,ME,2020-05-01,0.0,0.0,8.15217,30.0153,30.0153,668.0,0.0,0.0449331,0.0,0.0449331,[],[]
6,NH,2020-05-01,18.0104,0.0,12.7217,105.662,105.662,584.5,0.0,0.180773,0.0,0.180773,"[(""MA"", 18.0104)]",[]
7,NJ,2020-05-01,0.0,0.0,1073.99,5462.78,5462.78,4553.25,909.526,1.19975,909.526,1.19975,[],[]
8,NY,2020-05-01,0.0,0.0,1716.21,6059.07,6059.07,10406.2,0.0,0.582253,0.0,0.582253,[],[]
9,PA,2020-05-01,0.0,0.0,423.688,2699.62,2699.62,7980.25,0.0,0.338287,0.0,0.338287,[],[]


In [20]:
s = "NJ"
filter(row -> row.location == s, results.complete_table)

Row,location,date,sent,received,new_patients,active_patients,active_patients_nosent,capacity,overflow,load,overflow_nosent,load_nosent,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float32,Float64,Float64,Float32,Float32,Any,Any
1,NJ,2020-05-01,0.0,0.0,1073.99,5462.78,5462.78,4553.25,909.526,1.19975,909.526,1.19975,[],[]
2,NJ,2020-05-02,0.0,0.0,1055.75,5787.86,5787.86,4553.25,1234.61,1.27115,1234.61,1.27115,[],[]
3,NJ,2020-05-03,503.403,0.0,1036.88,6118.26,6118.25,4553.25,1565.01,1.34371,1565.0,1.34371,"[(""NY"", 503.403)]",[]
4,NJ,2020-05-04,0.0,30.9879,1018.0,5978.84,6451.25,4553.25,1425.59,1.31309,1898.0,1.41685,[],"[(""DE"", 30.9879)]"
5,NJ,2020-05-05,0.0,0.0,997.133,6315.63,6788.04,4553.25,1762.38,1.38706,2234.79,1.49081,[],[]
6,NJ,2020-05-06,0.0,30.8977,971.688,6678.72,7120.23,4553.25,2125.47,1.4668,2566.98,1.56377,[],"[(""DE"", 30.8977)]"
7,NJ,2020-05-07,617.656,0.0,950.161,7011.22,7452.74,4553.25,2457.97,1.53983,2899.49,1.6368,"[(""MA"", 617.656)]",[]
8,NJ,2020-05-08,0.0,0.0,926.771,6724.35,7783.52,4553.25,2171.1,1.47682,3230.27,1.70944,[],[]
9,NJ,2020-05-09,0.0,385.344,904.766,7440.76,8114.59,4553.25,2887.51,1.63417,3561.34,1.78215,[],"[(""PA"", 385.344)]"


In [21]:
results.sent_to

Dict{String, Vector{String}} with 10 entries:
  "NH" => ["CT", "MA", "RI"]
  "CT" => ["DE", "MA", "ME", "NH", "NJ", "NY", "PA"]
  "RI" => ["NJ", "NY"]
  "MA" => ["NJ", "NY", "RI"]
  "ME" => ["CT", "RI"]
  "NY" => ["CT", "DE"]
  "NJ" => ["CT", "MA", "MD", "NY", "RI"]
  "DE" => ["MD", "NJ", "NY"]
  "PA" => ["CT", "DE", "MD", "NJ", "NY"]
  "MD" => ["DE", "NJ", "NY", "PA"]